In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# initialize the dataframe

df = pd.read_csv('housing.csv')
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [3]:
# see available features and their data types

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [4]:
# check physical constraints and remove invalid observations

print(len(df[df['total_bedrooms'] > df['total_rooms']]))
print(len(df[df['households'] > df['population']]))

df = df[df['households'] <= df['population']]

0
3


In [5]:
# check string formatting of ocean_proximity

df['ocean_proximity'] = df['ocean_proximity'].apply(lambda x: x.lower())

df['ocean_proximity'].value_counts()

<1h ocean     9135
inland        6549
near ocean    2658
near bay      2290
island           5
Name: ocean_proximity, dtype: int64

In [6]:
# check for duplicates

df.duplicated().value_counts()

False    20637
dtype: int64

In [7]:
df[df['total_bedrooms'].notna()].describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20430.000000,20430.000000,20430.000000,20430.000000,20430.000000,20430.000000,20430.000000,20430.000000,20430.000000
mean,-119.570604,35.632999,28.635291,2636.812188,537.928879,1425.145032,499.494714,3.871544,206864.417719
std,2.003659,2.136235,12.591329,2185.275419,421.386931,1133.173287,382.292417,1.899135,115436.466637
min,-124.350000,32.540000,1.000000,2.000000,1.000000,5.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1450.250000,296.000000,788.000000,280.000000,2.563925,119500.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.537150,179700.000000
75%,-118.010000,37.720000,37.000000,3143.000000,647.000000,1722.750000,604.000000,4.744000,264700.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [8]:
df[df['total_bedrooms'].isna()].describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,207.000000,207.000000,207.000000,207.000000,0.0,207.000000,207.000000,207.000000,207.000000
mean,-119.472560,35.497633,29.270531,2562.603865,NaN,1477.772947,510.024155,3.822244,206007.280193
std,2.001424,2.097298,11.964927,1787.269789,NaN,1057.448212,386.120704,1.955595,111638.214545
min,-124.130000,32.660000,4.000000,154.000000,NaN,37.000000,16.000000,0.852700,45800.000000
25%,-121.810000,33.970000,19.000000,1307.500000,NaN,781.000000,258.000000,2.564150,128750.000000
50%,-118.490000,34.200000,30.000000,2155.000000,NaN,1217.000000,427.000000,3.411500,175000.000000
75%,-117.985000,37.495000,38.000000,3465.000000,NaN,1889.500000,628.000000,4.615750,267700.000000
max,-114.590000,40.920000,52.000000,11709.000000,NaN,7604.000000,3589.000000,15.000100,500001.000000


In [9]:
# data seems to be MCAR so we can drop missing values

df = df[df['total_bedrooms'].notna()]

In [10]:
# consider avg number of rooms/bedrooms held by each household on a block

df['avg_rooms_per_hh'] = df['total_rooms'] / df['households']
df['avg_bedrooms_per_hh'] = df['total_bedrooms'] / df['households']

df = df[df['avg_rooms_per_hh'] >= 1]
df = df[df['avg_bedrooms_per_hh'] >= 1]

In [11]:
# use property age as a categorical feature

df['housing_median_age_cat'] = df['housing_median_age'] >= 50

df['housing_median_age_cat'] = df['housing_median_age_cat'].apply(lambda x: 'old' if x == True else 'new')

In [12]:
df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,avg_rooms_per_hh,avg_bedrooms_per_hh
count,16079.000000,16079.000000,16079.000000,16079.000000,16079.000000,16079.000000,16079.000000,16079.000000,16079.000000,16079.000000,16079.000000
mean,-119.565153,35.653273,28.174140,2812.684184,585.095715,1504.176068,532.761552,3.730367,204942.910940,5.494061,1.135164
std,2.012152,2.156342,12.751387,2336.696736,449.784983,1206.852349,409.068915,1.844916,116414.766371,2.724052,0.528457
min,-124.350000,32.540000,1.000000,2.000000,1.000000,5.000000,1.000000,0.499900,14999.000000,1.000000,1.000000
25%,-121.780000,33.940000,18.000000,1540.500000,323.000000,823.000000,295.000000,2.485850,116450.000000,4.443662,1.034116
50%,-118.490000,34.270000,28.000000,2250.000000,474.000000,1222.000000,434.000000,3.375000,177100.000000,5.244300,1.067416
75%,-118.010000,37.740000,37.000000,3319.500000,702.000000,1817.000000,643.000000,4.546900,262500.000000,6.094575,1.116887
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000,141.909091,34.066667


In [13]:
# reorder the columns

cols = df.columns.tolist()
cols = cols[:2] + cols[3:8] + cols[9:] + [cols[8]]
df = df[cols]

df.columns

Index(['longitude', 'latitude', 'total_rooms', 'total_bedrooms', 'population',
       'households', 'median_income', 'ocean_proximity', 'avg_rooms_per_hh',
       'avg_bedrooms_per_hh', 'housing_median_age_cat', 'median_house_value'],
      dtype='object')

In [14]:
# export cleaned data as a csv

df.to_csv('housing_cleaned.csv', index=False)